In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import torch.utils.data as loader
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary

In [0]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [0]:
in_vect_size = 200

In [25]:
class generator(nn.Module):
    """
    Generator for 3D-GAN-
    The generator consists of five fully convolution layers with numbers of channels
    {512, 256, 128, 64, 1}, kernel sizes {4, 4, 4, 4, 4}, and strides {1, 2, 2, 2, 2}. 
    We add ReLU and batch normalization layers between convolutional layers, and a Sigmoid
    layer at the end. The input is a 200-dimensional vector, and the output is a 64 × 64 × 64 
    matrix with values in [0, 1].
    """
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.ConvTranspose3d(in_channels=200,out_channels=512,kernel_size=3,stride=1),
            nn.BatchNorm3d(512),
            nn.ReLU(),
        )
        self.layer2=nn.Sequential(
            nn.ConvTranspose3d(in_channels=512,out_channels=256,kernel_size=3,stride=2),
            nn.BatchNorm3d(256),
            nn.ReLU(),
        )
        self.layer3=nn.Sequential(
            nn.ConvTranspose3d(in_channels=256,out_channels=128,kernel_size=3,stride=2),
            nn.BatchNorm3d(128),
            nn.ReLU(),
        )
        self.layer4=nn.Sequential(
            nn.ConvTranspose3d(in_channels=128,out_channels=64,kernel_size=3,stride=2),
            nn.BatchNorm3d(64),
            nn.ReLU(),
        )
        self.layer5=nn.Sequential(
            nn.ConvTranspose3d(in_channels=64,out_channels=1,kernel_size=4,stride=2),
            nn.Sigmoid(),
        )
    
    def forward(self,x):
        bsize=x.size(0)
        x=x.view(bsize,in_vect_size,1,1,1)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.layer5(x)
        return x
G_=generator().to(device)
summary(G_,(1,in_vect_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose3d-1         [-1, 512, 3, 3, 3]       2,765,312
       BatchNorm3d-2         [-1, 512, 3, 3, 3]           1,024
              ReLU-3         [-1, 512, 3, 3, 3]               0
   ConvTranspose3d-4         [-1, 256, 7, 7, 7]       3,539,200
       BatchNorm3d-5         [-1, 256, 7, 7, 7]             512
              ReLU-6         [-1, 256, 7, 7, 7]               0
   ConvTranspose3d-7      [-1, 128, 15, 15, 15]         884,864
       BatchNorm3d-8      [-1, 128, 15, 15, 15]             256
              ReLU-9      [-1, 128, 15, 15, 15]               0
  ConvTranspose3d-10       [-1, 64, 31, 31, 31]         221,248
      BatchNorm3d-11       [-1, 64, 31, 31, 31]             128
             ReLU-12       [-1, 64, 31, 31, 31]               0
  ConvTranspose3d-13        [-1, 1, 64, 64, 64]           4,097
          Sigmoid-14        [-1, 1, 64,

In [33]:
class discriminator(nn.Module):
    """
    Discriminator for 3D-GAN-
    As a mirrored version of the generator, the discriminator takes as input a 
    64 × 64 × 64 matrix, and outputs a real number in [0, 1]. The discriminator 
    consists of 5 volumetric convolution layers, with numbers of channels {64,128,256,512,1}, 
    kernel sizes {4,4,4,4,4}, and strides {2, 2, 2, 2, 1}. There are leaky ReLU layers of 
    parameter 0.2 and batch normalization layers in between, and a Sigmoid layer at the end.
    """
    def __init__(self):
        super().__init__()
        self.layer1=nn.Sequential(
            nn.Conv3d(in_channels=1,out_channels=64,kernel_size=4,stride=2),
            nn.BatchNorm3d(64),
            nn.LeakyReLU(0.2)
        )
        self.layer2=nn.Sequential(
            nn.Conv3d(in_channels=64,out_channels=128,kernel_size=4,stride=2),
            nn.BatchNorm3d(128),
            nn.LeakyReLU(0.2)
        )
        self.layer3=nn.Sequential(
            nn.Conv3d(in_channels=128,out_channels=256,kernel_size=4,stride=2),
            nn.BatchNorm3d(256),
            nn.LeakyReLU(0.2)
        )
        self.layer4=nn.Sequential(
            nn.Conv3d(in_channels=256,out_channels=512,kernel_size=4,stride=2),
            nn.BatchNorm3d(512),
            nn.LeakyReLU(0.2)
        )
        self.layer5=nn.Sequential(
            nn.Conv3d(in_channels=512,out_channels=1,kernel_size=2,stride=1),
            nn.Sigmoid()
        )

    def forward(self,x):
        bsize=x.size(0)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.layer5(x)
        x=x.view(bsize,1)
        return x
D_=discriminator().to(device)
summary(D_,(1,64,64,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 64, 31, 31, 31]           4,160
       BatchNorm3d-2       [-1, 64, 31, 31, 31]             128
         LeakyReLU-3       [-1, 64, 31, 31, 31]               0
            Conv3d-4      [-1, 128, 14, 14, 14]         524,416
       BatchNorm3d-5      [-1, 128, 14, 14, 14]             256
         LeakyReLU-6      [-1, 128, 14, 14, 14]               0
            Conv3d-7         [-1, 256, 6, 6, 6]       2,097,408
       BatchNorm3d-8         [-1, 256, 6, 6, 6]             512
         LeakyReLU-9         [-1, 256, 6, 6, 6]               0
           Conv3d-10         [-1, 512, 2, 2, 2]       8,389,120
      BatchNorm3d-11         [-1, 512, 2, 2, 2]           1,024
        LeakyReLU-12         [-1, 512, 2, 2, 2]               0
           Conv3d-13           [-1, 1, 1, 1, 1]           4,097
          Sigmoid-14           [-1, 1, 